In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset=pd.read_csv('/kaggle/input/churn-predictions-personal/Churn_Predictions.csv')
dataset.head()

In [ ]:
X=dataset.iloc[:,3:-1].values
Y=dataset.iloc[:,-1].values

In [ ]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(missing_values=np.nan,strategy='mean')
X[:,4:12]=imputer.fit_transform(X[:,4:12])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=1)

In [ ]:
X_train

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,StandardScaler
gender=LabelEncoder()
gender_x=gender.fit(X_train[:,2])

X_train[:,2]=gender_x.transform(X_train[:,2])
X_test[:,2]=gender_x.transform(X_test[:,2])

country=LabelEncoder()
country_x=country.fit(X_train[:,1])
X_train[:,1]=country_x.transform(X_train[:,1])
X_test[:,1]=country_x.transform(X_test[:,1])

In [ ]:
X_train

In [ ]:
country_onehot=OneHotEncoder()
country_onehot_x=country_onehot.fit(X_train[:,1].reshape(-1,1))
country_onehot_x_train=country_onehot_x.transform(X_train[:,1].reshape(-1,1)).toarray()
country_onehot_x_test=country_onehot_x.transform(X_test[:,1].reshape(-1,1)).toarray()

X_train=np.concatenate((country_onehot_x_train,X_train),axis=1)
X_train=np.delete(X_train,4,axis=1)
X_test=np.concatenate((country_onehot_x_test,X_test),axis=1)
X_test=np.delete(X_test,4,axis=1)

In [ ]:
X_train.shape

In [ ]:
X_sc=StandardScaler()
X_sc=X_sc.fit(X_train)
X_train=X_sc.transform(X_train)
X_test=X_sc.transform(X_test)

In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

classifier= keras.models.Sequential()
classifier.add(Dense(units=7,activation='relu',input_shape=(12,)))
classifier.add(Dense(units=64,activation='relu'))
classifier.add(Dense(units=128,activation='relu'))
classifier.add(Dense(units=128,activation='relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units=1,activation='sigmoid'))

classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
classifier.summary()

In [ ]:
history=classifier.fit(X_train,Y_train,batch_size=64,epochs=50,verbose=1,validation_data=(X_test,Y_test),validation_batch_size=16)

In [ ]:
accu=history.history['accuracy']
loss=history.history['loss']
val_accu=history.history['val_accuracy']
val_loss=history.history['val_loss']

import matplotlib.pyplot as plt
plt.plot(accu,'r',label='Accuracy')
plt.plot(loss,'b',label='Loss')
plt.legend()
plt.figure()

plt.plot(val_accu,'r',label='Val_accuracy')
plt.plot(val_loss,'b',label='Val_loss')
plt.legend()
plt.figure()

plt.show()

**Prediction**<br>
Whether a person having,<br>
Geography: France<br>
Credit Score: 600<br>
Gender: Male<br> 
Age: 40 years old<br>
Tenure: 3 years<br>
Balance:  60000 <br>
Number of Products: 2 <br>
Does he have a credit card ? Yes<br>
Is this customer an Active Member: Yes<br>
Estimated Salary: 50000<br>

Will get the Loan?

In [ ]:
new_data=np.array([1,0,0,600,1,40,3,6000,2,1,1,50000])
new_data

In [ ]:
N_data=X_sc.transform(new_data.reshape(1,-1))

In [ ]:
prediction=classifier.predict(N_data)
if prediction>0.5:
    print('Approved')
else:
    print('Disapproved')
